In [1]:
from mp_api.client.mprester import MPRester
from pymatgen.core.periodic_table import Element
import pandas as pd 
import numpy as np 
import os

In [2]:
API_KEY = "fqPPo7Czb5mkbFh8mltlZd0I33csuKv0"
mpr = MPRester(API_KEY)

In [7]:
# version of MP database
mpr.get_database_version()

'2023.11.1'

In [8]:
help(mpr.summary)

Help on SummaryRester in module mp_api.client.routes.summary object:

class SummaryRester(mp_api.client.core.client.BaseRester)
 |  SummaryRester(*args, **kwds)
 |  
 |  Base client class with core stubs.
 |  
 |  Method resolution order:
 |      SummaryRester
 |      mp_api.client.core.client.BaseRester
 |      typing.Generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  search(self, band_gap: Union[Tuple[float, float], NoneType] = None, chemsys: Union[str, List[str], NoneType] = None, crystal_system: Union[emmet.core.symmetry.CrystalSystem, NoneType] = None, density: Union[Tuple[float, float], NoneType] = None, deprecated: Union[bool, NoneType] = None, e_electronic: Union[Tuple[float, float], NoneType] = None, e_ionic: Union[Tuple[float, float], NoneType] = None, e_total: Union[Tuple[float, float], NoneType] = None, efermi: Union[Tuple[float, float], NoneType] = None, elastic_anisotropy: Union[Tuple[float, float], NoneType] = None, elements: Union[List[str], NoneTyp

In [11]:
# docs=mpr.summary.search(elements=['Li', 'O'], all_fields=False, fields=['material_id', 'composition_reduced', 'formula_pretty','band_gap', 'symmetry', 'chemsys', 'structure'])
# docs=mpr.summary.search(chemsys="Li-Mn-O", all_fields=False, fields=["formula_pretty"])

Retrieving SummaryDoc documents:   0%|          | 0/482 [00:00<?, ?it/s]

In [19]:
from itertools import combinations

# 生成符合要求的化学系统 chemsys
# 首先必须要有氧化还原对，根据ceder关于无序岩盐的结构的综述（review-001）
# 主要氧化还原对有 (Ni2+/Ni3+, Ni2+/Ni4+), (Mn3+/Mn4+, Mn4+/Mn5+), (Fe2+/Fe3+, Fe3+/Fe4+), (V3+/V5+, V4+/V5+)，(Mo3+/Mo5+, Mo3+/Mo6+, Mo4+/Mo6+)，Ru4+/Ru5+，(Cr3+/Cr4+, Cr3+/Cr6+)
# 已报导的chemsystem(见review-001):Li–Ni–Mn–O, Li–Ni–Ti–O, Li–Mn-Ti–O, Li–Ti–Fe–O, Li-Ni-Ru-O, Li-Mn-V-O-F, Li-Mn-Nb-O-F

redox_ele = ["Ni", "Mn", "Fe", "V", "Mo", "Ru", "Cr", "Co"]
functional_ele = ['Ti', "Nb", "Cu", "Rh", "Zn", "Y"]

chemsys_list = []
# single redox element
for i in redox_ele:
    single_redox = "-".join(["Li", i, "O"])
    single_redox_f = "-".join(["Li", i, "O", "F"])
    chemsys_list.append(single_redox)
    chemsys_list.append(single_redox_f)
    for j in functional_ele:
        redox_functional = "-".join(['Li', i, j, "O"])
        redox_functional_f = "-".join(['Li', i, j, "O", "F"])
        chemsys_list.append(redox_functional)
        chemsys_list.append(redox_functional_f)
        
        
# double redox element 
for i in iter(combinations(redox_ele,2)):
    double_redox = "-".join(["Li", i[0], i[1], "O"])
    double_redox_f = "-".join(["Li", i[0], i[1], "O", "F"])
    chemsys_list.append(double_redox)
    chemsys_list.append(double_redox_f)
    for j in functional_ele:
        dredox_functional = "-".join(['Li', i[0], i[1], j, "O"])
        dredox_functional_f = "-".join(['Li', i[0], i[1], j, "O", "F"])
        chemsys_list.append(dredox_functional)
        chemsys_list.append(dredox_functional_f)

len(chemsys_list)  

504

In [60]:
# 根据chemsys_list来查询mp中相关的所有结构
docs=mpr.summary.search(chemsys=chemsys_list, all_fields=False,  num_sites=(20, 100), energy_above_hull=(0, 0.1), fields=["material_id", "formula_pretty", "nsites", "symmetry", "formation_energy_per_atom", "is_stable"])

Retrieving SummaryDoc documents:   0%|          | 0/3495 [00:00<?, ?it/s]

In [61]:
len(docs)

3495

In [63]:
df = pd.DataFrame(columns=['material_id', 'formula_pretty', 'nsites', 'crystal_system', 'formation_energy_per_atom', 'is_stable'])
for i in docs:
    data=[i.material_id, i.formula_pretty, i.nsites, i.symmetry.symbol, i.formation_energy_per_atom, i.is_stable]
    df.loc[len(df)]=data

In [64]:
df[:10]

,material_id,formula_pretty,nsites,crystal_system,formation_energy_per_atom,is_stable
0,mp-1178123,Li(CoO2)2,28,Cmc2_1,-1.430144,False
1,mp-1178332,Li(CoO2)2,56,P1,-1.457210,False
2,mp-1377260,Li(CoO2)2,56,P4_322,-1.457917,False
3,mp-25383,Li(CoO2)2,56,Fd-3m,-1.451605,False
4,mp-755386,Li(CoO2)2,28,Pmmn,-1.475349,False
5,mp-758122,Li(CoO2)2,28,Pbcn,-1.452094,False
6,mp-773076,Li(CoO2)2,56,P4_332,-1.486767,False
7,mp-774082,Li(CoO2)2,56,P2_1,-1.504020,True
8,mp-1285961,Li(CoO2)3,20,C2/m,-1.346426,False
9,mp-1307826,Li(CoO2)3,20,C2/m,-1.336444,False


In [65]:
df.to_csv('./materials.csv')
